In [2]:
from IPython.display import display, HTML
import matplotlib
import matplotlib.colors as mcolors
import pandas as pd
import numpy as np
import json

def extract_groups(data):
    # Handles both old and new structures
    groups = []
    if isinstance(data, dict) and 'groups_details' in data:
        groups = data['groups_details']
    elif isinstance(data, list):
        # List of investigations, each with groups_details
        for inv in data:
            if isinstance(inv, dict) and 'groups_details' in inv:
                groups.extend(inv['groups_details'])
    return groups


with open("result.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
print('Results file loaded successfully.')

if 'data' in locals():
    groups = extract_groups(data)
    if not groups:
        print("No groups to display.")
    else:
        # Only process groups with inves_type == 'vel_coeffs'
        for group in groups:
            if group.get('inves_type') == 'vel_coeffs':
                group_id = group.get('id', group.get('group_id', group.get('group', 'N/A')))
                print(f"\nGroup: {group_id}")
                # Print metadata (all keys except 'exps_details')
                meta = {k: v for k, v in group.items() if k != 'exps_details'}
                print("Metadata:", meta)
                # Prepare table from exps_details key, extracting nested fields
                exps_details = group.get('exps_details', [])
                if not exps_details:
                    print("No exps_details in this group.")
                    continue
                rows = []
                for exp in exps_details:
                    pso = exp.get('pso_params', {})
                    res = exp.get('results', {})
                    row = {
                        'w_inertia': pso.get('w_inertia', None),
                        'c_personal': pso.get('c_personal', None),
                        'c_global': pso.get('c_global', None),
                        'c_social': pso.get('c_social', None),
                        'avg_training_cost': res.get('avg_training_cost', None)
                    }
                    rows.append(row)
                table_df = pd.DataFrame(rows)
                # Use the specified fields and order
                required_cols = ['w_inertia', 'c_personal', 'c_global', 'c_social', 'avg_training_cost']
                missing = [col for col in required_cols if col not in table_df.columns]
                if missing:
                    print(f"Missing columns in exps_details: {missing}")
                    continue
                table_df = table_df[required_cols].copy()
                table_df['avg_training_cost'] = table_df['avg_training_cost'].round(2)
                # Sort by best cost (ascending, best at top)
                table_df = table_df.sort_values('avg_training_cost', ascending=True).reset_index(drop=True)
                n = len(table_df)
                num_bins = min(10, max(3, int(np.ceil(np.log2(n) + 1))))
                try:
                    bins = pd.qcut(table_df['avg_training_cost'], q=num_bins, retbins=True, duplicates='drop')
                    table_df['cost_bin'] = bins[0].cat.codes
                    bin_labels = bins[0].cat.categories
                except Exception:
                    bins = pd.cut(table_df['avg_training_cost'], bins=num_bins, retbins=True, duplicates='drop')
                    table_df['cost_bin'] = bins[0].cat.codes
                    bin_labels = bins[0].cat.categories
                cmap = matplotlib.cm.get_cmap('viridis', num_bins)
                bin_colors = [mcolors.to_hex(cmap(i/(num_bins-1))) for i in range(num_bins)]
                table_df['bin_color'] = table_df['cost_bin'].apply(lambda b: bin_colors[int(b)])
                bin_range_map = {i: str(bin_labels[i]) for i in range(len(bin_labels))}
                table_df['bin_range'] = table_df['cost_bin'].map(bin_range_map)
                table_df['color_swatch'] = table_df['bin_color'].apply(lambda c: f'<div style="width:30px;height:15px;background:{c};border:1px solid #888"></div>')
                # Rename columns for clarity
                table_df = table_df.rename(columns={
                    'w_inertia': 'inertia',
                    'c_personal': 'personal coeff',
                    'c_global': 'global coeff',
                    'c_social': 'social coeff',
                    'avg_training_cost': 'best cost',
                    'bin_range': 'bin (range)',
                    'bin_color': 'bin color',
                    'color_swatch': 'color swatch'
                })
                display(HTML(table_df[['inertia', 'personal coeff', 'global coeff', 'social coeff', 'best cost', 'bin (range)', 'bin color', 'color swatch']].to_html(escape=False, index=False)))
else:
    print("No data loaded.")


Results file loaded successfully.

Group: social-range
Metadata: {'inves_type': 'vel_coeffs', 'id': 'social-range', 'metadata': {'name': 'Social coeffs effect', 'description': 'Social coeffs effect'}}


C:\Users\61411\AppData\Local\Temp\ipykernel_35340\4284543610.py:76: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = matplotlib.cm.get_cmap('viridis', num_bins)


inertia,personal coeff,global coeff,social coeff,best cost,bin (range),bin color,color swatch
0.7,1.4,1.4,0.6,0.23,"(0.229, 0.29]",#440154,
0.7,1.4,1.4,0.8,0.29,"(0.229, 0.29]",#440154,
0.7,1.4,1.4,1.0,0.40,"(0.29, 0.4]",#31688e,
0.7,1.4,1.4,1.2,0.52,"(0.4, 0.52]",#35b779,
0.7,1.4,1.4,1.4,0.59,"(0.52, 0.59]",#fde725,


In [4]:
# Draw parallel coordinates for w_inertia, c_personal, c_social, c_global
# Each line represents a row from the previous table, colored by best cost (darker = better)
# If two lines coincide, the darker color (lower cost) should be drawn on top
import plotly.graph_objects as go
import pandas as pd

def plot_parallel_coords(table_df):
    # Use only the required columns
    axes = ['inertia', 'personal coeff', 'global coeff', 'social coeff']
    color_col = 'bin color'
    cost_col = 'best cost'
    # Sort so that lower cost (darker) is plotted last (on top)
    # Viridis: lower cost = lower bin = darker color
    # To ensure darker lines are on top, plot lightest (highest cost) first, darkest (lowest cost) last
    table_df_sorted = table_df.sort_values(cost_col, ascending=False).reset_index(drop=True)
    fig = go.Figure()
    for i, row in table_df_sorted.iterrows():
        y = [row[a] for a in axes]
        color = row[color_col]
        fig.add_trace(go.Scatter(
            x=axes,
            y=y,
            mode='lines+markers',
            line=dict(color=color, width=3),
            marker=dict(color=color, size=8),
            name=f"best_cost={row[cost_col]:.2f}",
            showlegend=False
        ))
    fig.update_layout(
        title="Parallel Coordinates: PSO Velocity Coefficients (colored by best cost)",
        xaxis_title="Parameter",
        yaxis_title="Value",
        template="plotly_white"
    )
    fig.update_xaxes(type='category')
    fig.show()

# Try to find the last table_df in scope (from previous cell)
if 'table_df' in locals() and not table_df.empty:
    plot_parallel_coords(table_df)
else:
    print("Run the previous cell to generate the table first.")
